In [124]:
# Setup for Google Colab directories
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')
%cd MLexercise_CarDimensions

# import glob
# import re
import pandas as pd
dataframes = pd.read_csv("data/cleaned_data.csv")
print(dataframes.columns.tolist())
dataframes

/content/drive/MyDrive/Colab Notebooks/MLexercise_CarDimensions
['MAKE', 'MODEL', 'Model Year', 'Overall Length', 'Overall Width', 'Overall Height', 'Wheelbase', 'Curb Weight', 'Front End Length', 'Rear End Length', 'Side Glass Height', 'Body Side Height', 'Roof Width', 'Front Overhang', 'Rear Overhang', 'Track Width Front', 'Track Width Rear', 'Weight Distribution', 'Front Weight %', 'Rear Weight %']


,MAKE,MODEL,Model Year,Overall Length,Overall Width,Overall Height,Wheelbase,Curb Weight,Front End Length,Rear End Length,Side Glass Height,Body Side Height,Roof Width,Front Overhang,Rear Overhang,Track Width Front,Track Width Rear,Weight Distribution,Front Weight %,Rear Weight %
0,FIAT,124SPIDER2DRCONVERTIBLE,16,407.0,173.0,124.0,231.0,1114.0,155.0,79.0,31.0,67.0,116.0,85.0,91.0,152.0,150.0,54/46,54.0,46.0
1,RAMTRUCKS,1500CREWCAB57FTBOX2WD,18,592.0,208.0,197.0,367.0,2255.0,130.0,321.0,45.0,97.0,140.0,101.0,124.0,174.0,173.0,57/43,57.0,43.0
2,RAMTRUCKS,1500CREWCAB57FTBOX2WD,18,592.0,202.0,197.0,367.0,2255.0,130.0,321.0,45.0,97.0,140.0,101.0,124.0,174.0,173.0,57/43,57.0,43.0
3,RAMTRUCKS,1500CREWCAB57FTBOX4X4,18,592.0,208.0,197.0,367.0,2327.0,130.0,321.0,45.0,97.0,140.0,101.0,124.0,174.0,173.0,57/43,57.0,43.0
4,RAMTRUCKS,1500CREWCAB57FTBOX4X4,18,592.0,202.0,197.0,367.0,2327.0,130.0,321.0,45.0,97.0,140.0,101.0,124.0,174.0,173.0,57/43,57.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3759,BMW,Z4SDRIVE35I35IS2DRCONVERTIBLERWD,9,424.0,179.0,129.0,250.0,1610.0,156.0,73.0,24.0,74.0,125.0,86.0,89.0,151.0,156.0,50/50,50.0,50.0
3760,BMW,Z4SDRIVE35I35IS2DRCONVRWD,9,424.0,179.0,129.0,250.0,1585.0,156.0,73.0,24.0,74.0,125.0,86.0,89.0,151.0,156.0,50/50,50.0,50.0
3761,BMW,Z4SDRIVE35I35IS2DRCONVRWD,9,424.0,179.0,129.0,250.0,1590.0,156.0,73.0,24.0,74.0,125.0,86.0,89.0,151.0,156.0,50/50,50.0,50.0
3762,ACURA,ZDX4DRSUVAWDTECHELITE,10,488.0,199.0,160.0,275.0,2064.0,133.0,109.0,33.0,88.0,120.0,109.0,104.0,172.0,171.0,58/42,58.0,42.0


In [125]:
# # Features and target variable
features = [
'Overall Length', 'Overall Width', 'Overall Height',
'Front End Length', 'Rear End Length',
'Side Glass Height', 'Body Side Height',
'Wheelbase', 'Front Overhang', 'Rear Overhang', # Sum = Overall Length
'Roof Width', 'Track Width Front', 'Track Width Rear', # ~ Overall Width
]

target = 'Curb Weight'

X = dataframes[features]
y = dataframes[target]


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.5, random_state=0)

In [128]:
# Model layers and training
# from tensorflow.keras.regularizers import l2

model = Sequential([
    Dense(512, activation='relu',
          input_shape=(X_train.shape[1],)
          ),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),

    # Dense(512, activation='relu',
    #       input_shape=(X_train.shape[1],),
    #       kernel_regularizer=l2(0.001)
    #       ),
    # Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    # Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    # Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(1)  # output layer for regression
])

# from tensorflow.keras.optimizers import Adam
# optimizer = Adam(learning_rate=.01)
# model.compile(optimizer=optimizer, loss='mse')
model.compile(optimizer='adam', loss='mse') # Default learning_rate=0.001
# model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(
    monitor='val_loss', patience=15, restore_best_weights=True
)
model.fit(
    X_train, y_train,
    epochs=500,
    batch_size=32,
    validation_split=0.2,  # 20% of training data used for validation
    callbacks=[early_stop],
    verbose=1)

Epoch 1/500


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 3183226.5000 - val_loss: 709956.3125
Epoch 2/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 512615.3438 - val_loss: 250982.8125
Epoch 3/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 175312.2188 - val_loss: 132974.5000
Epoch 4/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 98433.9297 - val_loss: 103461.9922
Epoch 5/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 83655.5938 - val_loss: 71244.2188
Epoch 6/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 63390.7266 - val_loss: 63734.1328
Epoch 7/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 56419.3789 - val_loss: 54515.3789
Epoch 8/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 54815.6133 - val_loss: 48462.1172
Epoch 9/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 46943.8516 - val_loss: 43765.3281
Epoch 10/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 49113.8906 - val_loss: 39984.7695
Epoch 11/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 36389.7070 -

In [129]:
y_pred = model.predict(X_test)
# print(y_pred)

59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [130]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

print("R² Score:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("Mean target weight:", y.mean())
print("Mean target predicted:", y_pred.mean())
mape = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", mape * 100, "%")


R² Score: 0.8607392336572999
MSE: 22988.31470310388
Mean target weight: 1808.4378320935175
Mean target predicted: 1813.8948
MAPE: 5.858413770470471 %


In [131]:
from notebooks.utils import compare_predictions

comparison = compare_predictions(y_test, y_pred)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#   print(comparison.head(100))

In [ ]:
y_pred_full = model.predict(X_scaled)

In [ ]:
full_comparison = compare_predictions(y, y_pred_full)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  print(full_comparison.head(100))
full_comparison.to_csv("results/weight_prediction_TF.csv", index=False)